# Multilabel Classification Using the ChestX-ray14 Dataset

## Step 0: Install PyHealth

In [1]:
!git clone https://github.com/EricSchrock/PyHealth.git
!cd PyHealth && git checkout ChestX-ray14 && pip install -e .

Cloning into 'PyHealth'...
remote: Enumerating objects: 8126, done.
remote: Counting objects: 100% (1718/1718), done.
remote: Compressing objects: 100% (530/530), done.
remote: Total 8126 (delta 1503), reused 1199 (delta 1188), pack-reused 6408 (from 2)
Receiving objects: 100% (8126/8126), 113.90 MiB | 16.80 MiB/s, done.
Resolving deltas: 100% (5260/5260), done.
Branch 'ChestX-ray14' set up to track remote branch 'ChestX-ray14' from 'origin'.
Switched to a new branch 'ChestX-ray14'
Obtaining file:///content/PyHealth
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pyhealth (pyproject.toml) ... done
  Created wheel for pyhealth: filename=pyhealth-2.0a8-py3-none-any.whl size=10674 sha256=958c7e0bd8938910e22eda0840e62272710f8cae2e42ad8531f1012a34cd222f
  Stored in di

## Step 1: Load Dataset

In [1]:
from pyhealth.datasets import ChestXray14Dataset

dataset = ChestXray14Dataset(download=True, partial=True)
dataset.stats()

INFO:pyhealth.datasets.chestxray14:Downloading ./images_01.tar.gz...


Checking MD5 checksum for ./images_01.tar.gz...


INFO:pyhealth.datasets.chestxray14:Checking MD5 checksum for ./images_01.tar.gz...


Extracting ./images_01.tar.gz...


INFO:pyhealth.datasets.chestxray14:Extracting ./images_01.tar.gz...


...

INFO:pyhealth.datasets.base_dataset:Scanning table: chestxray14 from /content/chestxray14-metadata-pyhealth.csv


INFO:pyhealth.datasets.base_dataset:Collecting global event dataframe...


Collected dataframe with shape: (4999, 26)


INFO:pyhealth.datasets.base_dataset:Collected dataframe with shape: (4999, 26)


Dataset: ChestX-ray14
Dev mode: False
Number of patients: 1335
Number of events: 4999


## Step 2: Define Task

In [2]:
samples = dataset.set_task()

Setting task ChestXray14MultilabelClassification for ChestX-ray14 base dataset...


INFO:pyhealth.datasets.base_dataset:Setting task ChestXray14MultilabelClassification for ChestX-ray14 base dataset...


Generating samples with 1 worker(s)...


INFO:pyhealth.datasets.base_dataset:Generating samples with 1 worker(s)...
Generating samples for ChestXray14MultilabelClassification with 1 worker: 100%|██████████| 1335/1335 [00:00<00:00, 1475.55it/s]

Label labels vocab: {'atelectasis': 0, 'cardiomegaly': 1, 'consolidation': 2, 'edema': 3, 'effusion': 4, 'emphysema': 5, 'fibrosis': 6, 'hernia': 7, 'infiltration': 8, 'mass': 9, 'nodule': 10, 'pleural_thickening': 11, 'pneumonia': 12, 'pneumothorax': 13}



INFO:pyhealth.processors.label_processor:Label labels vocab: {'atelectasis': 0, 'cardiomegaly': 1, 'consolidation': 2, 'edema': 3, 'effusion': 4, 'emphysema': 5, 'fibrosis': 6, 'hernia': 7, 'infiltration': 8, 'mass': 9, 'nodule': 10, 'pleural_thickening': 11, 'pneumonia': 12, 'pneumothorax': 13}
Processing samples: 100%|██████████| 4999/4999 [01:18<00:00, 63.31it/s]

Generated 4999 samples for task ChestXray14MultilabelClassification



INFO:pyhealth.datasets.base_dataset:Generated 4999 samples for task ChestXray14MultilabelClassification


In [3]:
from pyhealth.datasets import get_dataloader, split_by_sample

train_dataset, val_dataset, test_dataset = split_by_sample(samples, [0.7, 0.1, 0.2])

train_loader = get_dataloader(train_dataset, batch_size=16, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=16, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=16, shuffle=False)

## Step 3: Define Model

In [4]:
from pyhealth.models import CNN

model = CNN(dataset=samples)

/content/PyHealth/pyhealth/metrics/calibration.py:122: SyntaxWarning: invalid escape sequence '\c'
  accuracy of 1. Thus, the ECE is :math:`\\frac{1}{3} \cdot 0.49 + \\frac{2}{3}\cdot 0.3=0.3633`.


## Step 4: Train Model

In [5]:
from pyhealth.trainer import Trainer

# Only measure accurancy because with the "partial" dataset it is likely that
# there are not positive samples of every label present in the validation and test sets
trainer = Trainer(model=model, metrics=["accuracy"])
trainer.train(train_dataloader=train_loader, val_dataloader=val_loader, epochs=1)

CNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (cnn): ModuleDict(
    (image): CNNLayer(
      (cnn): ModuleList(
        (0): CNNBlock(
          (conv1): Sequential(
            (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (downsample): Sequential(
            (0): Conv2d(1, 128, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool2d(output_size=1)
    )
  )
  (fc): Linear(in_features=128, out_features=1

INFO:pyhealth.trainer:CNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (cnn): ModuleDict(
    (image): CNNLayer(
      (cnn): ModuleList(
        (0): CNNBlock(
          (conv1): Sequential(
            (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (downsample): Sequential(
            (0): Conv2d(1, 128, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool2d(output_size=1)
    )
  )
  (fc): Linear(in_featur

Metrics: ['accuracy']


INFO:pyhealth.trainer:Metrics: ['accuracy']


Device: cuda


...


INFO:pyhealth.trainer:--- Eval epoch-0, step-219 ---


accuracy: 0.9553


INFO:pyhealth.trainer:accuracy: 0.9553


loss: 0.1695


INFO:pyhealth.trainer:loss: 0.1695


## Step 5: Evaluate Model

In [6]:
trainer.evaluate(test_loader)

Evaluation: 100%|██████████| 63/63 [00:04<00:00, 14.36it/s]


{'accuracy': 0.9500714285714286, 'loss': 0.17985984730342078}